# Multi-Armed Bandits

The most important feature distinguishing reinforcement learning from other types of
learning is that it uses training information that evaluates the actions taken rather
than instructs by giving correct actions.

In [2]:
import numpy as np
import gymnasium as gym

import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Gymnasium

Gym is an open source Python library for developing and comparing reinforcement learning algorithms by providing a standard API to communicate between learning algorithms and environments, as well as a standard set of environments compliant with that API. Since its release, Gym's API has become the field standard for doing this.

Gymnasium is a maintained fork of OpenAI’s Gym library. The Gymnasium interface is simple, pythonic, and capable of representing general RL problems, and has a compatibility wrapper for old Gym environments.


## K-armed bandit problem implementation

In [ ]:
class KArmedBandit(gym.Env):

    def __init__(self, nb_arms=10, nb_steps=1000, mean=0, variance=1, noise_variance=1):
        self._nb_arms = nb_arms
        self._nb_steps = nb_steps

        self._mean = 0
        self._noise_mean = 0
        self._variance = variance
        self._noise_variance = noise_variance

        self.action_space = gym.spaces.Discrete(nb_arms)
        self.observation_space = gym.spaces.Discrete(1)
    
    def step(self, action):
        self._step += 1
    
        reward = self._arms[action]
        reward_noise = self.np_random.normal(self._noise_mean, self._noise_variance)
        terminated = self._step >= self._nb_steps

        info = { "is_optimal_action": int(action == np.argmax(self._arms)) }

        return reward + reward_noise, terminated, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self._step = 0
        self._arms = self.np_random.normal(self._mean, self._variance, size=self._nb_arms)